In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import os
import glob
import json
from sklearn.externals import joblib

In [3]:
# with open('oct20_nov01_2.json') as data:
#    data = json.load(data)
# print (data[:1])

In [4]:
# tweets = pd.DataFrame(data)
# print(tweets.shape)
# print(tweets[:3])

In [5]:
# tweets = pd.DataFrame(columns=['fullname','user','text','timestamp','likes','retweets','replies'])
# print (tweets.head())

In [6]:
path_to_json = "data/"
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

['nov01_dec01.json', 'jan01_feb01.json', 'dec01_jan01.json', 'oct14_oct20.json', 'oct20_nov01.json', 'feb01_mar01.json', 'oct01_oct14.json']


In [ ]:
data = []
for file in json_files:
    with open('data/' + file) as file:
        data.append(json.load(file))
#         file = json.load(file)
#         print(data.shape)
#         merge.append(data)
        # print('appending ' + file + ' to tweets ...')

In [ ]:
print(data[0:10])

In [ ]:
with open(json_files) as data:
   data = json.load(data)
print (data.shape)

In [ ]:
read_files = glob.glob("data/*.json")
with open("merged_file.json", "wb") as outfile:
    outfile.write('[{}]'.format(
        ','.join([open(f, "rb").read() for f in read_files])))

In [ ]:
tweets = pd.DataFrame(columns=['fullname','user','text','timestamp','likes','retweets','replies'])
print (tweets.head())

In [ ]:
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        fullname = json_text['fullname']
        text = json_text['text']
#         lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [fullname, text]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)

In [ ]:
print(tweets.head(10))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# cv = CountVectorizer()
# X_cv = cv.fit_transform(tweets.text)
# # cv.get_feature_names() # prints the list of the tokenized words 
# # X_cv.toarray() # prints the array of each tweet and a 1 if it includes the word in that spot
# cv.vocabulary_.get('metoo') # the converse mapping from feature name to column index
# # print(X_cv.shape)
# # print(X_cv[0:1])

# # joblib.dump(cv, 'cv.pkl')

In [ ]:
# bv = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
# X_bv = bv.fit_transform(tweets.text)

# # bv.get_feature_names() # prints the list of the tokenized words 
# # X_bv.toarray() # prints the array of each tweet and a 1 if it includes the word in that spot
# bv.vocabulary_.get('metoo') # the converse mapping from feature name to column index
# # print(X_bv.shape)
# # print(X_bv[0:1])

# # joblib.dump(bv, 'bv.pkl')

In [ ]:
hv = HashingVectorizer(ngram_range=(1,2), stop_words='english')
# hv
X_hv = hv.transform(tweets.text)
# print(X_hv.shape)
# print(X_hv[0:1])

# joblib.dump(hv, 'hv.pkl')

In [ ]:
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_hv)
print(X_tfidf.shape)
print(X_tfidf[0:1])

joblib.dump(transformer, 'transformer.pkl')

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(tweets.text)

In [ ]:
# terms = tfidf_vectorizer.get_feature_names()
# terms = transformer.get_feature_names()

In [ ]:
print(type(cv))
print(type(X_tfidf))

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(X_tfidf)

In [ ]:
y_kmeans = kmeans.predict(X_tfidf)

In [ ]:
print(kmeans.cluster_centers_)

In [ ]:
print(kmeans.labels_.shape)

clusters = kmeans.labels_.tolist()
print(clusters[:10])

In [ ]:
joblib.dump(kmeans, 'kmeans.pkl')

In [ ]:
tweets['cluster'] = clusters
# print(tweets.head())
cluster_0 = tweets[tweets.cluster == 0][['fullname','text','timestamp','user']]
cluster_1 = tweets[tweets.cluster == 1][['fullname','text','timestamp','user']]
cluster_2 = tweets[tweets.cluster == 2][['fullname','text','timestamp','user']]
# print(cluster_0.head(3))

In [ ]:
cluster_0.to_csv('cluster_0')
cluster_1.to_csv('cluster_1')
cluster_2.to_csv('cluster_2')

In [ ]:
# frame['cluster'].value_counts()

In [ ]:
# dictionary = {
#     'text': tweets.text, 
#     'user': tweets.user, 
#     'timestamp': tweets.timestamp,
#     'likes': tweets.likes,
#     'cluster': clusters
# }
# frame = pd.DataFrame(dictionary, 
#                      index = [clusters] , 
#                      columns = ['user', 'text', 'timestamp', 'likes', 'cluster'])

In [ ]:
# frame['cluster'].value_counts()

In [ ]:
# print(frame.head())

In [ ]:
# frame.loc[frame['cluster'] == 0]

In [ ]:
# cluster_1.to_csv('cluster_1')

In [ ]:
# grouped = frame['likes'].groupby(frame['cluster'])
# print(grouped.type())

In [ ]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % tweets.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
#     print("Cluster %d titles:" % i, end='')
#     for user in frame.ix[i]['user'].values.tolist():
#         print(' %s,' % user, end='')
#     print() #add whitespace
#     print() #add whitespace
    
print()
print()